In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import mysql.connector as sql
import math
import itertools
from functools import reduce
import statsmodels.formula.api as smf
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set()

In [4]:
business_df = pd.read_csv('../Dataset/Project3Data/yelp_business.csv')
category_list = pd.read_csv('../Dataset/Project3Data/yelp-business-categories-list.csv')

In [6]:
category_list.drop_duplicates(subset='Secondary Categories', keep="first", inplace=True)

def pd_to_dict(df):
    category_dict = {k: g["Secondary Categories"].tolist() for k,g in df.groupby("Primary Categories")}
    category_dict = dict((v,k) for k in category_dict for v in category_dict[k])
    return category_dict

category_dict = pd_to_dict(category_list)

In [7]:
def get_primary_category_label(df):
    df.categories = df.categories.apply(lambda x:x.rstrip('\r')).str[0:].str.split(';')
    df.categories = df.categories.apply(lambda col: col[0])
    return df

get_primary_category_label(business_df).sample(3, random_state=42)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
21976,PBZyumZ_0vSijc2zODdqgQ,"""Bowl Tique Avenue""",Spring Valley,"""4075 S Durango Dr, Ste 106""",Las Vegas,NV,89147,36.115402,-115.280675,4.5,59,0,Restaurants
64788,Fygp6fXxdFvnP5zRLtCDeA,"""Augustina Boutiques""",Summer Hill,"""3 MacPherson Avenue""",Toronto,ON,M5R 1A6,43.678899,-79.390810,4.5,3,1,Fashion
141978,n6xiAKcTu_uIRzrUIwiB0w,"""Full Force Fitness""",NaN,"""3130 E Union Hills Dr, Ste 109""",Phoenix,AZ,85050,33.655613,-112.015468,4.5,7,1,Active Life


In [8]:
def map_main_category(df, dict_):
    df['primary_category'] = df['categories'].map(dict_)
    #df.drop(['categories'], axis =1, inplace=True)
    return df

tableau_business = map_main_category(business_df, category_dict)
tableau_business.fillna('Other', inplace=True)

In [9]:
tableau_business.to_csv('tableau_business.csv')